<a href="https://colab.research.google.com/github/himanshu20032/sampleproject/blob/master/CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch
from tqdm import tqdm

In [2]:
data = pd.read_csv('/content/drive/MyDrive/CF_DATA/u.data',delimiter='\t',names=['user_id','item_id','rating','time'])

In [3]:
data = data.drop('time',axis=1)

In [4]:
movie_matrix = pd.DataFrame(index = list(set(data['user_id'])), columns = list(set(data['item_id'])),dtype=float)
movie_matrix = movie_matrix.fillna(0.0)
for i in data.index:
  user = data.iloc[i]['user_id']
  item = data.iloc[i]['item_id']
  rating = data.iloc[i]['rating']
  movie_matrix.loc[user][item] = rating
  

In [5]:
himanshu = movie_matrix.loc[:10,:8].copy()

In [6]:
#himanshu.to_csv('hi.csv',header=True,index=True)

In [7]:
np.array(himanshu.loc[:8,1])

array([5., 4., 0., 0., 4., 4., 0., 0.])

In [8]:
#cosine_similarity([np.array(himanshu.loc[:8,1])], [np.array(himanshu.loc[:8,2])])

In [9]:
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [10]:
def get_default_device():
  if torch.cuda.is_available():
    return torch.device('cuda')
  else :
    return torch.device("cpu")
device = get_default_device()

In [11]:
from sklearn.model_selection import KFold

## ITEM BASED

In [12]:
#trainning
train_data = himanshu.loc[1:8,:].copy()
test_data = himanshu.loc[9:,:].copy()

def finding_similarity(data,train_data): 
  similarity_matrix = pd.DataFrame(index = list(data.columns), columns = list(data.columns), dtype=float )
  similarity_matrix = similarity_matrix.fillna(0.0)
  for i in similarity_matrix.index:
    similarity_matrix.loc[i][i] = 1.0
  
  for item in train_data.index:
    for i in train_data.index:
      if i != item :
        a = np.array(train_data.loc[:,item]).reshape(1,-1)
        a = torch.from_numpy(a).float().to(device)
        
        b = np.array(train_data.loc[:,i]).reshape(1,-1)
        b = torch.from_numpy(b).float().to(device)
        #cosine_similarity(a,b)
        cos = torch.nn.CosineSimilarity(dim=1)
        temp=cos(a,b)
        similarity_matrix.loc[item][i] = torch.Tensor.numpy(temp.to('cpu'))[0]      
  return similarity_matrix


In [13]:
similarity_matrix = finding_similarity(himanshu,train_data)

In [14]:
similarity_matrix

,1,2,3,4,5,6,7,8
1,1.000000,0.744845,0.585206,0.301086,0.585206,0.585206,0.445964,0.379257
2,0.744845,1.000000,0.707107,0.363803,0.707107,0.707107,0.384900,0.109109
3,0.585206,0.707107,1.000000,0.514496,1.000000,1.000000,0.544331,0.154303
4,0.301086,0.363803,0.514496,1.000000,0.514496,0.514496,0.863506,0.740959
5,0.585206,0.707107,1.000000,0.514496,1.000000,1.000000,0.544331,0.154303
6,0.585206,0.707107,1.000000,0.514496,1.000000,1.000000,0.544331,0.154303
7,0.445964,0.384900,0.544331,0.863506,0.544331,0.544331,1.000000,0.776927
8,0.379257,0.109109,0.154303,0.740959,0.154303,0.154303,0.776927,1.000000


In [15]:
a=np.array(similarity_matrix.loc[1])

In [16]:
b=np.array(test_data.loc[10])

In [17]:
sum = sum(a)

In [18]:
a = np.array([1,2,3,4])
a = a[::-1]
a[:2]

array([4, 3])

NameError: ignored

In [29]:
def similarity_of_topn_movies(movie,similarity_matrix,topn):
  #similarity = np.array(similarity_matrix.loc[movie])
  temp = []
  for i in similarity_matrix.columns:
    temp.append((similarity_matrix.loc[movie][i],i))
  temp = sorted(temp,key=lambda x: x[0])
  index = []
  similarity = []
  for a in temp :
    similarity.append(a[0])
    index.append(a[1])
  similarity.pop()
  index.pop()
  similarity = similarity[::-1]
  similarity = similarity[:topn]
  index = index[::-1]
  index = index[:topn]
  return np.array(similarity),index

In [23]:
def rating_of_topn_movies(user,test_data,topn_index):
  rating = []
  for i in topn_index:
    rating.append(test_data.loc[user][i])
  return np.array(rating)


In [38]:
def item_prediction_error( similarity_matrix , test_data , topn):
  error=0.0
  for user in test_data.index :
    prediction = []
    actual = test_data.loc[user]
    for movie in test_data.columns :
      if test_data.loc[user][movie] :
        #similarity = np.array(similarity_matrix.loc[movie])
        similarity,topn_index = similarity_of_topn_movies(movie,similarity_matrix,topn)
        total_similarity = np.sum(similarity);
        #rating = np.array(test_data.loc[user])
        rating = rating_of_topn_movies(user,test_data,topn_index)
        similarity = torch.from_numpy(similarity).float().to(device)
        rating = torch.from_numpy(rating).float().to(device)
        kitty = torch.Tensor.numpy(torch.sum(torch.mul(similarity,rating).to('cpu')) ) /total_similarity
        prediction.append(int(kitty))
      else:
        prediction.append(0.0)
    #print(prediction-actual)
    result = prediction - actual
    for kim in result:
      error+=abs(kim)
  return error



In [40]:

item_prediction_error(similarity_matrix,test_data,8)

19.0

In [28]:
 similarity_of_topn_movies(1,similarity_matrix,3)

(array([0.74484527, 0.58520573, 0.58520573, 0.58520573, 0.44596392,
        0.37925667, 0.30108586]), [2, 6, 5, 3, 7, 8, 4])

## USER BASED

In [54]:
top_similar_user(10,1,train_data,test_data,0.0)

[0. 0. 0. 4. 0. 0. 4. 0.]
[0. 3. 4. 3. 3. 5. 4. 1.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0.]
[0. 3. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 2. 4.]
[0. 0. 0. 5. 0. 0. 5. 5.]
[0. 0. 0. 0. 0. 0. 3. 0.]


([1, 6, 7, 8], [0.5368755, 0.31622776, 0.81649655, 0.7071068])

In [56]:
def top_similar_user(user_id,movie,train_data,test_data,threshold):
  u1 = np.array(test_data.loc[user_id])
  u1[movie-1] = 0
  #print(u1)
  u1 = u1.reshape(1,-1)
  u1 = torch.from_numpy(u1).float().to(device)
  similarity = []
  similar_user = []
  for other_user in train_data.index:
    cos = torch.nn.CosineSimilarity(dim=1)
    u2 = np.array(train_data.loc[other_user])
    u2[movie-1] = 0
    #print(u2)
    u2 = u2.reshape(1,-1)
    u2 = torch.from_numpy(u2).float().to(device)
    temp=cos(u1,u2)
    temp = torch.Tensor.numpy(temp.to('cpu'))[0]
    if temp > threshold: 
      similarity.append(temp)      
      similar_user.append(other_user)
  return similar_user,similarity

In [60]:
def user_prediction_error(train_data, test_data, threshold):
  error=0
  for user in tqdm(test_data.index):
    prediction = []
    actual = test_data.loc[user]
    for movie in test_data.columns:
      
      if test_data.loc[user][movie] :
        similar_user , similarity = top_similar_user(user,movie,train_data,test_data,threshold)
        similarity = np.array(similarity)
        sum_of_similarity = np.sum(similarity)
        similarity = torch.from_numpy(similarity).float().to(device)
        rating = []
        if len(similar_user) > 0 :
          for i in similar_user :
            rating.append(train_data.loc[i][movie])
          rating  = np.array(rating)
          rating = torch.from_numpy(rating).float().to(device)
          calculated  = torch.sum(rating * similarity)
          calculated = calculated.to('cpu')
          
          cal = torch.Tensor.numpy(calculated)/sum_of_similarity
          prediction.append(int(cal))
        else:
          prediction.append(0)
      else:
        prediction.append(0)
    #print(user," ",prediction)
    result = prediction - actual
    for kim in result:
      error+=abs(kim)
    #error += abs( np.sum(prediction - actual ))
  return error
  

In [ ]:
user_prediction_error(train_data, test_data, 0.0)

In [66]:
train_data = movie_matrix.loc[:800,:]
test_data = movie_matrix.loc[801:,:]

In [2]:
jovian.commit()

[jovian] Detected Colab notebook...


[jovian] Error: jovian.commit doesn't work on Colab unless the notebook was created and executed from Jovian.
Make sure to run the first code cell at the top after executing from Jovian.
Alternatively, you can download this notebook and upload it manually to Jovian.
Learn more: https://jovian.ai/docs/user-guide/run.html#run-on-colab


In [1]:
! pip install jovian --upgrade
import jovian


Requirement already up-to-date: jovian in /usr/local/lib/python3.6/dist-packages (0.2.32)
